# Finding h- and W-Theta at h-

In [1]:
# import the basics
import pysr
from pysr import PySRRegressor

import xarray as xr
import netCDF4 as nc
import h5netcdf
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from glob import glob

# sara also has
from netCDF4 import Dataset
import os

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
folder_path = '/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/'

nc_test1 = nc.Dataset(os.path.join(folder_path, 'Ug2Q010_I.nc'), mode='r')
z_256 = nc_test1.variables['z'][:]
zh_257 = nc_test1.variables['zh'][:]
t_120 = nc_test1.variables['time'][:]

nc_test2 = nc.Dataset(os.path.join(folder_path, 'Ug2Q010_IV.nc'), mode='r')
z_384 = nc_test2.variables['z'][:]
zh_385 = nc_test2.variables['zh'][:]
t_90 = nc_test2.variables['time'][:]
print(len(z_256), len(zh_257), len(t_120), len(z_384), len(zh_385), len(t_90))

256 257 120 384 385 90


In [3]:
def read_data(filepath):
    """
    Reads in each dataset and returns an xarray dataset
    """

    ds_thm = xr.open_dataset(f"{filepath}",
                             decode_times=False, group="thermo")
    ds_bud = xr.open_dataset(f"{filepath}",
                            decode_times=False, group="budget")
    ds_def = xr.open_dataset(f"{filepath}", 
                             decode_times=False, group="default")
    ds = xr.merge([ds_thm, ds_bud, ds_def], join="inner")

    if len(ds.time) == 90:
        ds = ds.assign_coords({"time": t_90}).swap_dims({"time":"time"})
    elif len(ds.time) == 120:
        ds = ds.assign_coords({"time": t_120}).swap_dims({"time":"time"})

    if len(ds.z) == 256:
        ds = ds.assign_coords({"z": z_256}).swap_dims({"z":"z"})
    elif len(ds.z) == 384:
        ds = ds.assign_coords({"z": z_384}).swap_dims({"z":"z"})

    if len(ds.zh) == 257:
        ds = ds.assign_coords({"zh": zh_257}).swap_dims({"zh":"zh"})
    elif len(ds.zh) == 385:
        ds = ds.assign_coords({"zh": zh_385}).swap_dims({"zh":"zh"})

    return ds

* Four inversion cases, i being the strong inversion and iv the weakest (temperature inversion that occurs at the top of the ABL)
* Surface Heating (Q): Influences the heat profile.
* Wind (Ug): Imposed wind; affects the movement and distribution of air properties.
* Inversion (dtheta/dz): Impacts the stratification and stability, influencing PBL growth.

In [4]:
# renaming files according to their inversion strength, wind, and surface heating
folder_path = '/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/'
files = glob(f"{folder_path}/*")

for file in files:
    print(file)

/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug16Q006_I.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug8Q006_IV.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug8Q003_IV.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug16Q010_IV.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug2Q024_I.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug8Q003_III.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug16Q001_IV.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug2Q01_III.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug16Q003_IV.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug2Q010_I.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug1Q01_II.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug8Q003_II.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug16Q000_IV.nc
/Users/gretavanzetten/Desktop/LEAPSummer2024/les_sim_2/Ug2Q010_IV.nc
/Users/gretavanzetten/Desktop/LE

In [5]:
# put all file names into one list for accessability later
file_names = [
    "Ug16Q006_I.nc", 
    "Ug8Q006_IV.nc", 
    "Ug8Q003_IV.nc",
    "Ug16Q010_IV.nc",
    "Ug2Q024_I.nc",
    "Ug8Q003_III.nc",
    "Ug16Q001_IV.nc",
    "Ug2Q01_III.nc",
    "Ug16Q003_IV.nc",
    "Ug2Q010_I.nc",
    "Ug1Q01_II.nc",
    "Ug8Q003_II.nc",
    #"Ug16Q000_IV.nc",
    "Ug2Q010_IV.nc",
    "Ug16Q006_IV.nc"
]

In [6]:
# put all the files into one dataset
datasets = [
    read_data(files[0]),
    read_data(files[1]),
    read_data(files[2]),
    read_data(files[3]),
    read_data(files[4]),
    read_data(files[5]),
    read_data(files[6]),
    read_data(files[7]),
    read_data(files[8]),
    read_data(files[9]),
    read_data(files[10]),
    read_data(files[11]),
    # no Ug16Q000_IV.nc
    read_data(files[13]),
    read_data(files[14])
]

In [7]:
# print short and long names of variables within each group 
### default - velocity terms
### thermo - buoyancy or theta
### budget - TKE or heat budget terms

df = nc.Dataset(files[10], mode='r')
for group_name, group in df.groups.items():
    print(f"Group: {group_name}")
    for var_name, var in group.variables.items():
        long_name = var.getncattr('long_name') if 'long_name' in var.ncattrs() else 'No long name'
        dimensions = var.dimensions if hasattr(var, 'dimensions') else 'No dimensions attribute'
        print(f"Variable: {var_name}, Long Name: {long_name}, Dimensions: {dimensions}")
    print("\n")
df.close()

Group: default
Variable: iter, Long Name: Iteration number, Dimensions: ('time',)
Variable: area, Long Name: Fractional area contained in mask, Dimensions: ('time', 'z')
Variable: areah, Long Name: Fractional area contained in mask, Dimensions: ('time', 'zh')
Variable: eft, Long Name: eft, Dimensions: ('time', 'z')
Variable: eft_3, Long Name: Moment 3 of the eft, Dimensions: ('time', 'z')
Variable: eft_4, Long Name: Moment 4 of the eft, Dimensions: ('time', 'z')
Variable: eft_diff, Long Name: Diffusive flux of the eft, Dimensions: ('time', 'zh')
Variable: eft_w, Long Name: Turbulent flux of the eft, Dimensions: ('time', 'zh')
Variable: eft_grad, Long Name: Gradient of the eft, Dimensions: ('time', 'zh')
Variable: eft_2, Long Name: Moment 2 of the eft, Dimensions: ('time', 'z')
Variable: eft_path, Long Name: eft path, Dimensions: ('time',)
Variable: eft_flux, Long Name: Total flux of the eft, Dimensions: ('time', 'zh')
Variable: sft, Long Name: sft, Dimensions: ('time', 'z')
Variable: s

### Helpful Variable Names
* pbl_height = 'zi' 
* mean potential temperature = 'th'
* heat_flux = 'th_flux' 
* time = 'time'

Want to look at $\theta$ flux and find the minimum $\rightarrow$ the minimum is h-

In [11]:
def time_average(data_array, timeavg):
    """
    Parameters:
    - data_array (xarray.DataArray),input data array with 'time' dimension
    - timeavg (int), number of time steps to average over

    Returns:
    - xarray.DataArray, time-averaged data array.
    """
    # make sure timeavg is positive int + smaller than the length of data
    if timeavg <= 0 or timeavg > data_array.sizes["time"]:
        raise ValueError("timeavg must be a positive integer less than or equal to the number of time steps in data.")

    # resample data array by averaging over specified intervals
    averaged_data_array = data_array.coarsen(time=timeavg, boundary="trim").mean()

    return averaged_data_array

In [29]:
def make_wtheta_hmin(path, items, time_avg):
    """
    Extracts the variables of interest from the LES simulations and averages them over specified number of time steps.
    Parameters:
    - path (str): The path to the LES simulation data.
    - items (list): A list of file names containing the LES simulation data. Ex: ['Ug16Q001_IV.nc', 'Ug16Q003_IV.nc', 'Ug16Q006_I.nc']
    - time_avg (int): The number of time steps over which to average.
    Returns:
    - time averaged wtheta_hmins
    - time averaged hmins
    """
    # Initialize empty lists to accumulate results
    hmin = []
    wtheta_hmin = []
    
    for item in items:
        hmin_temp = []
        wtheta_hmin_temp = []
        ds_stat = nc.Dataset(os.path.join(path, item), mode='r')
        wtheta = ds_stat.groups['thermo']['th_flux'][:]
        zh = ds_stat.variables['zh'][:]
        
        for t in range(wtheta.shape[0]):
            wtheta_at_t = wtheta[t]
            min_wtheta = np.min(wtheta_at_t)
            min_height = zh[np.argmin(wtheta_at_t)]
            hmin_temp.append(min_height)
            wtheta_hmin_temp.append(min_wtheta)
        
        # ---- added by chat to make sure passing a 'xarray.DataArray' into my 'time_average' function
        time_coords = np.arange(len(hmin_temp)) # first add time coordinates to 'time_average' 

        #convert numpy arrays to xarray.DataArray
        hmin_data_array = xr.DataArray(hmin_temp, dims=["time"], coords={"time": time_coords})
        wtheta_hmin_data_array = xr.DataArray(wtheta_hmin_temp, dims=["time"], coords={"time": time_coords})

        # Use the time_average function with xarray.DataArray
        hmin_temp_timeavg = time_average(hmin_data_array, time_avg)
        wtheta_hmin_temp_timeavg = time_average(wtheta_hmin_data_array, time_avg)

        # Append the results to the respective lists
        hmin.append(hmin_temp_timeavg)
        wtheta_hmin.append(wtheta_hmin_temp_timeavg)
    
    # Concatenate the results along the time axis
    hmin = np.concatenate(hmin, axis=0)
    wtheta_hmin = np.concatenate(wtheta_hmin, axis=0)
    return hmin, wtheta_hmin

In [34]:
hmin, wtheta_min = make_wtheta_hmin(folder_path, file_names, 15)
print("h-: ", hmin)
print("wtheta min LENGTH: ", len(wtheta_min))
print("wtheta min: ", wtheta_min)

h-:  [ 811.6  821.2  820.4  822.4  824.4  837.6  843.6  836.8  928.   955.6
  970.4  998.  1016.4 1022.8 1045.6 1062.   843.2  862.8  879.6  886.
  888.   912.   919.2  934.4  984.8 1028.4 1059.6 1100.  1122.8 1167.2
  869.6  885.6  892.   904.   913.6  933.6  952.8  963.6  802.4  820.4
  828.8  834.   843.6  859.2  860.8  865.6  801.2  814.4  836.4  802.
  840.   865.6  850.4  852.4  944.   962.   986.  1006.8 1017.2 1042.4
 1056.8 1062.8  880.   913.2  946.8  915.6  946.   968.4  983.2  982.4
  822.4  828.   836.   838.8  847.2  842.8  856.   856.   909.2  927.6
  950.   962.8  981.6  998.8 1011.6 1019.6  786.8  807.2  812.   815.2
  830.4  836.   844.8  847.2  970.4 1001.6 1017.6 1058.  1087.6 1108.8
  918.   947.6  978.4 1002.8 1010.  1042.8]
wtheta min LENGTH:  106
wtheta min:  [-0.02176679 -0.02402852 -0.0227475  -0.01909418 -0.01912026 -0.02194497
 -0.02516051 -0.02229727 -0.0096364  -0.01092938 -0.01163426 -0.01107084
 -0.01227187 -0.01105633 -0.0112859  -0.00898893 -0.00600461